# C-Interfaces

<ul id="top">
  <li><a href="#My-Structs">My Structs</a></li>
  <li><a href="#FCIRK-integrator">FCIRK-integrator</a></li>
  <li><a href="#Composition-integrator">Composition-integrator</a></li>
  <li><a href="#Binary-IO">Binary-IO</a></li>
</ul>

## My Structs

In [3]:
"""
FCIRK 
"""
struct ODEProblem
           Nkepler::Int64
           Moreq::Int64
           ode::Int64  
           tspan::Tuple{Float64,Float64}
           u0:: Array{BigFloat,1}
           k::  Array{BigFloat,1} 
           rpar:: Array{BigFloat,1}
           ipar::Array{Int32,1}
end

function ODEProblem(ode,tspan,u0,k,rpar;Nkepler=length(k),Moreq=0,ipar=[0])
        return ODEProblem(Nkepler,Moreq,ode,tspan,u0,k,rpar,ipar)
end;

"""
Composition and Splitting methods
"""
struct ODEProblem2
           neq::Int64
           ode::Int64
           tspan::Tuple{Float64,Float64}       
           u0:: Array{BigFloat,1}
           k:: Array{BigFloat,1} 
           rpar:: Array{BigFloat,1}
           ipar::Array{Int32,1}
end


function ODEProblem2(ode,tspan,u0,k,rpar;neq=length(u0),ipar=[0])
    return ODEProblem2(neq,ode,tspan,u0,k,rpar,ipar)
end;

In [2]:
"""
FCIRK 
"""
function solve(myfilename::String,
               prob:: ODEProblem,
               alg:: Function,
               h::Float64;
               ns::Int64=8, 
               saveat::Int64=1, 
               threads::Int64=1,        
               adaptive::Bool=false,   
               mixed::Bool=true)        

    if adaptive==false
        xadaptive=0;
    else 
        xadaptive=1;
    end
    
    if  mixed==false
        xmixed=0;
    else 
        xmixed=1;
    end
      
    alg(prob,ns,myfilename,h,saveat,threads,xadaptive,xmixed)
    
end;  


"""
Composition and Splitting methods
"""
function solve(   myfilename::String, 
                  prob:: ODEProblem2,
                  alg:: Function,
                  codmethod::Int64,
                  h::Float64;
                  saveat::Int64=1,
                  mixed::Bool=false) 
    
    if  mixed==false
        xmixed=0;
    else 
        xmixed=1;
    end
            
    alg(codmethod,xmixed,prob,myfilename,h,saveat)
    
end;    

<a href="#top">Back to the top</a>

## FCIRK integrator

In [3]:
#   FCIRK implementation:
#      doubleFCIRK
#      longFCIRK
#      quadFCIRK
#

In [4]:
function quadFCIRK( prob::ODEProblem,
                     ns:: Int64,
                     myfilename::String, 
                     h::Float64,
                     saveat::Int64, 
                     threads::Int64,   # Optional argument
                     adaptive::Int64,
                     mixed::Int64
    ) 
    #
    result_array=Array{Float64}(undef,10)
    
    ulen=length(prob.u0)
    klen=length(prob.k)
    rlen=length(prob.rpar)
    ilen=length(prob.ipar)
    
    errorcode=ccall(
     # name of C function and library
    (:quadFCIRK, "../../../Code/C/Code-FCIRK/libFCIRKQUAD.so"),
    # output type
     Cint,     
    # tuple of input types
    (Cint,Cint,Cint,Cdouble,Cdouble,                         
     Ptr{BigFloat},Cint,
     Cdouble,
     Ptr{BigFloat},Cint,
     Ptr{BigFloat},Cint,
     Ptr{Cint},Cint,
     Cstring,Cint, Cint,
     Cint,Cint,Cint,
     Ref{Cdouble}),
    # name of Julia variables to pass in
     prob.Nkepler,prob.Moreq,ns,prob.tspan[1],prob.tspan[2],   
     prob.u0,ulen,h,prob.k,klen,prob.rpar,rlen,prob.ipar,ilen,
     myfilename,saveat,
     prob.ode,
     threads,adaptive,mixed,
     result_array
    )
    if errorcode !=0 
        print("error at quad-FCIRK.c")
    end
    return result_array

end

quadFCIRK (generic function with 1 method)

In [18]:
function longFCIRK( prob::ODEProblem,
                     ns:: Int64,
                     myfilename::String, 
                     h::Float64,
                     saveat::Int64, 
                     threads::Int64,   # Optional argument
                     adaptive::Int64,
                     mixed::Int64
    ) 
    #
    result_array=Array{Float64}(undef,10)
    
    ulen=length(prob.u0)
    klen=length(prob.k)
    rlen=length(prob.rpar)
    ilen=length(prob.ipar)
        
    
    errorcode=ccall(
    # name of C function and library
    (:longFCIRK, "../../../Code/C/Code-FCIRK/libFCIRKLONG.so"),
    # output type
     Cint,  
    # tuple of input types
    (Cint,Cint,Cint,Cdouble,Cdouble,                         
     Ptr{BigFloat},Cint,
     Cdouble,
     Ptr{BigFloat},Cint,
     Ptr{BigFloat},Cint,
     Ptr{Cint},Cint,
     Cstring,Cint, Cint,
     Cint,Cint,Cint,
     Ref{Cdouble}),
    # name of Julia variables to pass in
     prob.Nkepler,prob.Moreq,ns,prob.tspan[1],prob.tspan[2],    
     prob.u0,ulen,h,prob.k,klen,prob.rpar,rlen,prob.ipar,ilen,
     myfilename,saveat,
     prob.ode,
     threads,adaptive,mixed,
     result_array
    )
    if errorcode !=0 
        print("error at long-FCIRK.c")
    end
    return result_array

end

longFCIRK (generic function with 1 method)

In [6]:
function doubleFCIRK( prob::ODEProblem,
                      ns:: Int64,
                      myfilename::String, 
                      h::Float64,
                      saveat::Int64, 
                      threads::Int64,   # Optional argument
                      adaptive::Int64,
                      mixed::Int64
    ) 
    #
    result_array=Array{Float64}(undef,10)  
    
    ulen=length(prob.u0)
    klen=length(prob.k)
    rlen=length(prob.rpar)
    ilen=length(prob.ipar)
    
    errorcode=ccall(
    # name of C function and library
    (:doubleFCIRK, "../../../Code/C/Code-FCIRK/libFCIRKDOUBLE.so"),
     # output type
     Cint,            
    # tuple of input types
    (Cint,Cint,Cint,Cdouble,Cdouble,                         
     Ptr{BigFloat},Cint,
     Cdouble,
     Ptr{BigFloat},Cint,
     Ptr{BigFloat},Cint,
     Ptr{Cint},Cint,
     Cstring,Cint, Cint,
     Cint,Cint,Cint,
     Ref{Cdouble}),
     # name of Julia variables to pass in
     prob.Nkepler,prob.Moreq,ns,prob.tspan[1],prob.tspan[2],   
     prob.u0,ulen,h,prob.k,klen,prob.rpar,rlen,prob.ipar,ilen,
     myfilename,saveat,
     prob.ode,
     threads,adaptive,mixed,
     result_array
    )
    
   if errorcode !=0 
        print("error at double-FCIRK.c")
    end
    return result_array

end

doubleFCIRK (generic function with 1 method)

<a href="#top">Back to the top</a>

## Composition integrator

In [11]:
#   Composition implementation: 
#       doubleComp
#       longComp
#       quadComp.

In [12]:
function longComp( codmethod::Int64,
                   mixed::Int64,
                   prob::ODEProblem2,
                   myfilename::String, 
                   h::Float64,
                   saveat::Int64) 
    #
    result_array=Array{Float64}(undef,10)
    
    ulen=length(prob.u0)
    klen=length(prob.k)
    rlen=length(prob.rpar)
    ilen=length(prob.ipar)
    
    errorcode=ccall(
    # name of C function and library
    (:longComp, "../../../Code/C/Code-Comp/libCompLONG.so"),
    # output type
     Cint, 
    # tuple of input types
    (Cint,Cint,Cdouble,Cdouble,                                                              
     Ptr{BigFloat},Cint,Cdouble,Ptr{BigFloat},Cint,
     Ptr{BigFloat},Cint,Ptr{Cint},Cint,
     Cint,Cstring,Cint,Cint,
     Ref{Cdouble}),
    # name of Julia variables to pass in
     prob.neq,codmethod,prob.tspan[1],prob.tspan[2],    
     prob.u0,ulen,h,prob.k,klen,
     prob.rpar,rlen,prob.ipar,ilen,
     mixed,myfilename,saveat,prob.ode,
     result_array
    )
    if errorcode !=0 
        print("error at long-Comp.c")
    end
    return result_array

end

longComp (generic function with 1 method)

In [13]:
function quadComp( codmethod::Int64,
                   mixed::Int64,
                   prob::ODEProblem2,
                   myfilename::String, 
                   h::Float64,
                   saveat::Int64) 
    #
    result_array=Array{Float64}(undef,10)
    
    ulen=length(prob.u0)
    klen=length(prob.k)
    rlen=length(prob.rpar)
    ilen=length(prob.ipar)
    
    errorcode=ccall(
    # name of C function and library
    (:quadComp, "../../../Code/C/Code-Comp/libCompQUAD.so"),   
    # output type
     Cint,
    # tuple of input types
    (Cint,Cint,Cdouble,Cdouble,                                                            
     Ptr{BigFloat},Cint,Cdouble,Ptr{BigFloat},Cint,
     Ptr{BigFloat},Cint,Ptr{Cint},Cint,
     Cint,Cstring,Cint,Cint,Ref{Cdouble}),
     # name of Julia variables to pass in
     prob.neq,codmethod,prob.tspan[1],prob.tspan[2],   
     prob.u0,ulen,h,prob.k,klen,
     prob.rpar,rlen,prob.ipar,ilen,
     mixed,myfilename,saveat,prob.ode,
     result_array
    )
    if errorcode !=0 
        print("error at quad-Comp.c")
    end

end

quadComp (generic function with 1 method)

In [14]:
function doubleComp( codmethod::Int64,
                   mixed::Int64,
                   prob::ODEProblem2,
                   myfilename::String, 
                   h::Float64,
                   saveat::Int64) 
    #
    result_array=Array{Float64}(undef,10)
    
    ulen=length(prob.u0)
    klen=length(prob.k)
    rlen=length(prob.rpar)
    ilen=length(prob.ipar)    
    
    errorcode=ccall(
    # name of C function and library
    (:doubleComp, "../../../Code/C/Code-Comp/libCompDOUBLE.so"),   
    # output type
     Cint,         
    # tuple of input types
    (Cint,Cint,Cdouble,Cdouble,                                                              
     Ptr{BigFloat},Cint,Cdouble,Ptr{BigFloat},Cint,
     Ptr{BigFloat},Cint,Ptr{Cint},Cint,
     Cint,Cstring,Cint,Cint,
     Ref{Cdouble}),
    # name of Julia variables to pass in
     prob.neq,codmethod,prob.tspan[1],prob.tspan[2],    
     prob.u0,ulen,h,prob.k,klen,
     prob.rpar,rlen,prob.ipar,ilen,
     mixed,myfilename,saveat,prob.ode,
     result_array
    )
    if errorcode !=0 
        print("error at long-Comp.c")
    end
    return result_array

end

doubleComp (generic function with 1 method)

<a href="#top">Back to the top</a>

## Binary-IO

In [15]:
#   Binary-Output
#       Readbin
#       Appendbin

In [1]:
function Readbin(nout::Integer,neq::Integer,myfilename::String,
                 ret=0::Int64   # Optional argument
                 )

    result_array=Vector{BigFloat}(undef,nout*(neq+1));     
    
     for i in 1:(nout*(neq+1)) 
       result_array[i]=BigFloat(0)
    end

    errorcode=ccall(
    # name of C function and library
    (:CReadBin, "../../../Code/C/Code-Binary/libCBinary.so"), 
    # output type
    Cint,       
    # tuple of input types
   (Cint,Cint,Cstring,Ptr{BigFloat}),
    # name of Julia variables to pass in
    nout, neq, myfilename,result_array                 
    )
    if errorcode !=0 
        error("error at CBinFilesWRA.c")
    end
    
    sol=transpose(reshape(result_array,neq+1,:))
    
    # Covert to sol as Array{Array{BigFloat,1},1}
    
    (rows,cols)=size(sol)
    ti=Vector{BigFloat}(undef,rows) 
    ui=Vector{Vector{BigFloat}}(undef,rows)
    
    if (ret==0)
        ti.=sol[:,1]
        for i in 1:rows
           ui[i]=sol[i,2:end]
        end       
        return ti,ui
    else
        return result_array
    end
end

Readbin (generic function with 2 methods)

In [2]:
function Appendbin(nout::Integer,neq::Integer,myfilename::String,
                   sol1::Array{Array{BigFloat,1},1})
      
    errorcode=ccall(
    # name of C function and library
    (:CAppendBin, "../../../Code/C/Code-Binary/libCBinary.so"),  
    # output type
     Cint,               
    # tuple of input types
    (Cint,Cint,Cstring,Ptr{BigFloat}),     
    # name of Julia variables to pass in
     nout,neq,myfilename,soli                           
     )
    if errorcode !=0 
        error("error at CBinFilesWRA.c")
    end
end

Appendbin (generic function with 1 method)

<a href="#top">Back to the top</a>